In [ ]:
!git clone https://github.com/anantgupta129/ERA-V1.git
%cd ERA-V1/easytorch/

In [ ]:
!python setup.py sdist
!pip install .

In [ ]:
%cd /content/

In [ ]:
import albumentations as A
import torch
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from torchvision import transforms as T

from easytorch.dataset import build_cifar
from easytorch.models.custom_resnet import ResNet18

# Load Data

In [ ]:
batch_size = 512

kwargs = {
    "batch_size": batch_size,
    "shuffle": True,
    "num_workers": 2,
    "pin_memory": True,
}
mean = (0.49139968, 0.48215841, 0.44653091)
std = (0.24703223, 0.24348513, 0.26158784)
train_transforms = A.Compose(
    [
        A.Sequential(
            [
                A.CropAndPad(px=4, keep_size=False),  # padding of 2, keep_size=True by defaulf
                A.RandomCrop(32, 32),
            ]
        ),
        A.CoarseDropout(1, 16, 16, 1, 16, 16, fill_value=mean, mask_fill_value=None),
        A.Normalize(mean, std),
        ToTensorV2(),
    ]
)
train_loader = build_cifar(set="train", augments=train_transforms, **kwargs)
test_loader = build_cifar(set="test", **kwargs)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [ ]:
model = ResNet18().to(device)
model.summarize(device, (1, 3, 32, 32))

In [ ]:
from torch.nn import CrossEntropyLoss

from easytorch.utils.misc import find_lr

model = ResNet18().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

kwargs = {"end_lr": 10, "num_iter": 200, "step_mode": "exp"}
find_lr(model, device, optimizer, criterion=CrossEntropyLoss(), dataloader=train_loader, **kwargs)

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR

from easytorch.trainer import Trainer

num_epochs = 20

scheduler = OneCycleLR(
    optimizer,
    max_lr=1.63e-02,
    steps_per_epoch=len(train_loader),
    epochs=num_epochs,
    pct_start=5 / num_epochs,
    div_factor=100,
    three_phase=False,
    final_div_factor=100,
    anneal_strategy="linear",
)

# scheduler = None
trainer = Trainer(model, device, optimizer, scheduler)
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}")
    trainer.train(train_loader)
    trainer.evaluate(test_loader)

In [ ]:
trainer.plot_history()

# Evaluate

In [ ]:
from easytorch.dataset import make_transform
from easytorch.utils.misc import per_class_accuracy
from easytorch.utils.plotting import plot_misclassified

test_transforms = make_transform("test")

In [ ]:
per_class_accuracy(model, device, test_loader)

In [ ]:
misclassified_imgs = plot_misclassified(
    model, test_loader, device, test_transforms, return_imgs=True
)

# GradCam Results

In [ ]:
model.layer4[1]

In [ ]:
from easytorch.utils.gradcam import plot_cam_on_image

plot_cam_on_image(model, [model.layer4[1]], misclassified_imgs, {"mean": mean, "std": std})